### Step-by-step computation of equivalent baselines

##### by Claudia

Given an array of horns, there is a certain number of baselines you can define. Afterwards, you can partitionate the set in subsets of equivalent baselines.

Let's work in a square array of $N=n \times n$ horns (for example, $n=8$ horns in a side of the array. Total number of horns: $N=64$)

##### by James

adding to this by finding the unique combinations of pairs of horns already identified and calculating the RMSE for each horn combination

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from itertools import combinations
import scipy.special
import qubic
from CSFPA_dataIO import calculate_intensity_4_baseline, IntegrateHornCombOnFP

import os
import re
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm
from timeit import default_timer as timer

In [ ]:
n=8

N= n*n

In [ ]:
print("n= ",n, "N= ",N)

### Let's define the coordinates of the horns

In [ ]:
# Which are the coordinates of these horns? In normalized units (unit= separation of two horns in one axis):

Coordinates_horns = []

count = 0
for i in range(n):
    for j in range(n):
        a = (i,j)
        print("Coordinates (x,y) are ", a)
        
        Coordinates_horns.append(a)
        
        count += 1

print("count: ", count)


In [ ]:
print(Coordinates_horns)

In [ ]:
Coordinates_horns = np.array(Coordinates_horns)
Coordinates_horns.shape

**Note**:  The $i$ horn has coordinates $(x_i,y_i)$, which are stored in the $x_i*n +y_i$ element of the **Coordinates_horns** array.

For example, for the horn with coordinates $(2,3)$, the position in the array is:


$2n +3 = 19$

We can take this number as the label of the horn.


### Now, let's compute the baselines

The number of baselines that you can construct with $N$ horns is ${N(N-1)} \over{2}$.

(This gives all posible combinations of two **different** horns, without repetition).

If we think of a $N \times N$ matrix with all the possible combinations, we would only take the upper (or lower) triangle.

In [ ]:
N_baselines = N*(N-1)/2

print(N_baselines)

 Each baseline can be thought of as a pseudo-vector, with a given length $L$ and a given orientation $\alpha$ (i.e., an oriented segment). To be able to compute $L$ and $\alpha$, we need to know the position of the horns that form the baseline.

Let's label somehow the baselines, using the label of the horns that constitute them.

If a baseline is formed with horns $i$ and $j$, let's take the upper triangle. Then if $i$ labels the row and $j$ labels the column, we will have: $j > i$.

So we do a loop over $i$ values, from $0$ to $N-1$, and then a nested loop over $j$ from $i+1$ to $N-1$.

For each, I have a baseline. I compute the $L^2$ and the $\tan (\alpha)$

$L^2= (x_i - x_j)^2 + ( y_i - y_j)^2 $

$\tan (\alpha) = (y_j - y_i)/(x_j - x_i)$

In [ ]:
# Check how many combinations we have:
count = 0
for i in range(N):
    for j in range(i+1,N):
        count = count +1
        print(count)

In [ ]:
# Let's test the coordinates of the horns that form a given baseline.

for i in range(N):
    for j in range(i+1,N):
        print("for the horn", i," the coordinates are: ", Coordinates_horns[i])
        print("for the horn", j," the coordinates are: ", Coordinates_horns[j])
        

In [ ]:
# For each baseline, let us compute L2 and tan_alpha:

baseline = []   #array that will keep L2 and the angle

baseline_v2 = []    #array that will keep the label of the horns that form the baseline, L2 and the angle

for i in range(N):
    x_i,y_i = Coordinates_horns[i]

    for j in range(i+1,N):
        
        x_j,y_j = Coordinates_horns[j]        


        L2 = (x_i - x_j)**2 + (y_i - y_j)**2
        
        tan_alpha = (y_j - y_i)/(x_j - x_i)
        
        angle= np.arctan(tan_alpha)
        
        baseline.append([L2, angle])
        
        baseline_v2.append([i,j, L2, angle])
        

baseline = np.array(baseline)

baseline_v2 = np.array(baseline_v2)


In [ ]:
# I order following L2 and then following angle. Then, I will need to separate them in subgroups to count
# how many there are in a given category.

from operator import itemgetter

ordered_baselines_v2 = sorted(baseline_v2, key= itemgetter(2,3))


ordered_baselines_v2 = np.array(ordered_baselines_v2)

In [ ]:
# Check that we have an ordered array (and we have the explicit number of the horns)

print(ordered_baselines_v2)

In [ ]:
# Another check to see if I'm getting what I want:

count = 0
for i in range(N):

    for j in range(i+1,N):      
        
        
        print(i,j, ordered_baselines_v2[count])
        
        count += 1
        
        
        

 ### Intermediate computing.
    
In the following, we do some computations, to separate the baselines in categories according to the value of $L2$ and **angle**.


This is inspired by the example in the next (commented) cell.

In [ ]:
# These are the L2 values:
ordered_baselines_v2[:,2]

In [ ]:
# I compute where the L2 value changes:
np.diff(ordered_baselines_v2[:,2])

In [ ]:
len(np.diff(ordered_baselines_v2[:,2]))

In [ ]:
xx = np.diff(ordered_baselines_v2[:,2])

In [ ]:
np.where(xx)[0]

In [ ]:
# These are the indices where the L2 value changes 
np.where(xx)[0]+1

In [ ]:
# I split the array in the positions where L2 changes:

zz= np.split(ordered_baselines_v2, np.where(np.diff(ordered_baselines_v2[:,2]))[0]+1)


In [ ]:
#Check:
np.shape(zz)

In [ ]:
# Now, for each distinct value of L2, I split the array in different values of the angle.

partitioned_baselines = []

for i in range(len(zz)):

    
    aa = zz[i]
    
    bb = np.split(aa, np.where(np.diff(aa[:,3]))[0]+1)

    bb = np.array(bb)
    
    partitioned_baselines.append(bb)
    

partitioned_baselines = np.array(partitioned_baselines)


In [ ]:
print(np.shape(partitioned_baselines))

print(len(partitioned_baselines), partitioned_baselines.shape)


In [ ]:
# for each value of L2, how many different values of the angle we have:

for i in range(len(partitioned_baselines)):
    print(len(partitioned_baselines[i]), partitioned_baselines[i])
    
    

In the following cell, I compute each unique baseline (characterized by a given value of $L^2$ and an given angle), and compute how many equivalent baselines there are in each category. 

If we want to make tests using equivalent baselines, we can read the corresponding horns' labels from the $0$ and $1$ elements of the **partitioned_baselines** array, and make the appropriate selection.

In [ ]:
# Now I compute the number of unique baselines, and for each of them, 
# I compute the number of equivalent baselines (i.e. how many elements there are in that category):

N_unique = 0

for i in range(len(partitioned_baselines)):

    n_angles  = len(partitioned_baselines[i])
    
    for j in range(n_angles):

        print(partitioned_baselines[i][j])
         
        print(" ")
        
        N_eq = len(partitioned_baselines[i][j])

        print(" Number of equivalent baselines for this particular baseline: ", N_eq)
        #print(" Shape of equivalent baseline set", partitioned_baselines[i][j].shape)
        print(" ")
        
        N_unique += 1

        
print("Number of unique baselines: ", N_unique)
        
    
    

In [ ]:
"""for each one of claudias baseline types
save for a file
produce auxilliary file"""

claudiasbaselinerep = "/home/james/mylibs/multifrequency/baseline_files/ClaudiasBaselines/"

In [ ]:
fmt = '%d', '%d', '%1.4f', '%1.4f'

N_unique = 0

for i in range(len(partitioned_baselines)):

    n_angles  = len(partitioned_baselines[i])
    
    for j in range(n_angles):

        print(partitioned_baselines[i][j])
         
        print(" ")
        
        N_eq = len(partitioned_baselines[i][j])

        print(" Number of equivalent baselines for this particular baseline: ", N_eq)
        print(" Shape of equivalent baseline set", partitioned_baselines[i][j].shape)
        print(" ")
        
        N_unique += 1
        
        index1_partitioned_baselines = partitioned_baselines[i][j][:,0:2] + 1
        index2_partitioned_baselines = np.hstack((index1_partitioned_baselines, partitioned_baselines[i][j][:,2:4]))
        print(index1_partitioned_baselines.shape, index2_partitioned_baselines)
        file = open(claudiasbaselinerep+"CBtype_"+str(N_unique)+".txt", "w")
        file.write("pairTDh1, pairTDh2, L2, angle"+'\n')
        np.savetxt(file, index2_partitioned_baselines, delimiter=', ', fmt=fmt)
        file.close()

test load the files just created
and test outputting the unique combinations for each type

In [ ]:
cbfiles = glob.glob(claudiasbaselinerep+'*.txt')
#print(len(cbfiles))

for file in cbfiles:
    print(file)
    dat = np.loadtxt(file, delimiter=',', skiprows=1)
    #print(len(dat.shape))
    
    if len(dat.shape) == 1:
        print("break for baseline of 1")
        continue
    
    bi = np.linspace(0, len(dat[:,0])-1, len(dat[:,0]), dtype=int)
    arsize = scipy.special.factorial(len(bi)) / ( scipy.special.factorial(2) * scipy.special.factorial(len(bi) - 2))
    #print("file, dat, bi, arsize, ", file, dat.shape, bi, arsize)
    comb = np.array(list(combinations(bi, 2)))
    #print(comb[0])
    
    for i in range(len(comb[:,0])):
        print(i, comb[i], comb[i,0], comb[i,1], 
              dat[comb[i,0], 0], dat[comb[i,0], 1], dat[comb[i,1], 0], dat[comb[i,1], 1])

use qubic libraries to relate TD numbers to FI numbers. This is because the optical simulation data is labelled by FI numbers

In [ ]:
"""above relates TD unique combinations
now setup FI numeric conversion"""
FIhorns = np.linspace(1,400,400, dtype=int)

d = qubic.qubicdict.qubicDict()
d.read_from_file('/home/james/libraries/qubic/qubic/dicts/pipeline_demo.dict')
d['config'] = 'FI'
q = qubic.QubicInstrument(d)

centers = q.horn.center[:, 0:2]
col = q.horn.column
row = q.horn.row

instFI = qubic.QubicInstrument(d)
hornsFI = instFI.horn.open

hornsTD = (col >= 8) & (col <= 15) & (row >= 8) & (row <= 15)

instTD = qubic.QubicInstrument(d)
instTD.horn.open[~hornsTD] = False

cnt = 1
TDhornsFIconf = np.zeros(400)

%matplotlib notebook
plt.figure(figsize=(10,10))
q.horn.plot()
for i in range(len(centers)):
    if hornsTD[i] == True:
        #plt.text(centers[i,0]-0.006, centers[i,1], 'c{0:}'.format(col[i]), color='r',fontsize=8)
        #plt.text(centers[i,0]+0.00001, centers[i,1], 'r{0:}'.format(row[i]), color='b',fontsize=8)
        plt.text(centers[i,0]-0.005, centers[i,1]-0.004, 'h {0:}'.format(str(i+1)), color='g',fontsize=8)
        plt.text(centers[i,0]-0.005, centers[i,1], 'td {0:}'.format(str(cnt)), color='r',fontsize=8)
        
        TDhornsFIconf[i] = cnt
        
        cnt+=1
instTD.horn.plot()
plt.ylabel('Horn GRF Y (m)')
plt.xlabel('Horn GRF X (m)')

plt.show()

In [ ]:
"""now convert TD to FI and check"""
for file in cbfiles:
    print(file)
    dat = np.loadtxt(file, delimiter=',', skiprows=1)
    #print(len(dat.shape))
    
    if len(dat.shape) == 1:
        print("break for baseline of 1")
        continue
    
    bi = np.linspace(0, len(dat[:,0])-1, len(dat[:,0]), dtype=int)
    arsize = scipy.special.factorial(len(bi)) / ( scipy.special.factorial(2) * scipy.special.factorial(len(bi) - 2))
    #print("file, dat, bi, arsize, ", file, dat.shape, bi, arsize)
    comb = np.array(list(combinations(bi, 2)))
    print(comb.shape)
    
    for i in range(len(comb[:,0])):
        print(i, comb[i], comb[i,0], comb[i,1], 
              dat[comb[i,0], 0], dat[comb[i,0], 1], dat[comb[i,1], 0], dat[comb[i,1], 1],
             int(FIhorns[np.where(TDhornsFIconf == dat[int(comb[i,0]),0])]),
             int(FIhorns[np.where(TDhornsFIconf == dat[int(comb[i,0]),1])]),
             int(FIhorns[np.where(TDhornsFIconf == dat[int(comb[i,1]),0])]),
             int(FIhorns[np.where(TDhornsFIconf == dat[int(comb[i,1]),1])]))

testing qubic libs working and a function to get RMSE calculations

In [ ]:
basedir='/home/james/libraries/qubic/qubic'
#basedir = Qubic_DataDir(datafile='instrument.py', ) 
print('basedir : ', basedir)
dictfilename = basedir + '/dicts/global_source_oneDet.dict'
d = qubic.qubicdict.qubicDict()
#d.read_from_file('../qubic/qubic/dicts/global_source_oneDet.dict')
#change to moddded dictionary
d.read_from_file('/home/james/libraries/qubic/qubic/dicts/global_source_oneDet.dict')
d['config'] = 'FI'
q = qubic.QubicMultibandInstrument(d)

vtxs = q[0].detector.vertex
vtxcounter = np.zeros(992)
print("vertexes shape: ", vtxs.shape)

def RMSEtested(v1,v2):
    return np.sqrt(np.mean((v1-v2)**2)), mean_squared_error(v1, v2, squared=False)

def RMSE(v1,v2):
    return np.sqrt(np.mean((v1-v2)**2))

from sklearn.metrics import mean_squared_error
#just to check...
#MSE = mean_squared_error(y_actual, y_predicted, squared=False)
 
#RMSE = math.sqrt(MSE)

here is a directory to the simulation files, Here is a link - https://maynoothuniversity-my.sharepoint.com/:u:/g/personal/james_murphy_2018_mumail_ie/EWFWx6YTco5ChvdzA1-MWesBTpRqsv30ttpKVwOlI8Ue1w?e=nKWAP6


In [ ]:
"""now calc RMSE for each baseline in combination
use in loop above"""
my150dat = '/media/james/DATA/GRASPdata/MyTabSourceFIModel/150GHz/MODALfiles/'
#also need vtxs

def RMSEcalc(file):

    #print(i)
    x, y, i1 = calculate_intensity_4_baseline((120, 146), my150dat)
    x, y, i2 = calculate_intensity_4_baseline((121, 147), my150dat)
    
    rmse = RMSE(i1,i2)
    print(rmse)
    #px, py, pi = IntegrateHornCombOnFP(i, np.array([x, y]), vtxs)
    #print(x.shape, i.shape, px.shape, pi.shape)
    return rmse

test parralel code

In [ ]:
"""time to calc 1 horn pair as FP baseline and then integrat on dets = 37 seconds \n
lets say 378 pair combs in baseline = 37 * 2 * 378 \n
approx 8 hours
for that baseline type

I should probably try this parrallel function again...!
"""

In [ ]:


num_cores=16
inputs=tqdm(cbfiles)

In [ ]:
# startm = timer()

# if __name__ == '__main__':
#         process=Parallel(n_jobs=num_cores)(delayed(RMSEcalc)(file) for file in inputs)
        
# endm = timer()
# print("time taken parrallel", endm - startm, "s")



In [ ]:
#print(len(process), len(cbfiles))

#cbfilesshort = cbfiles[:57]

#print(cbfilesshort)




cbfiles.sort(key=lambda f: int(re.sub('\D', '', f)))

print(cbfiles[95:108])

cbfilesshort = cbfiles[103:108]

# print(cbfilesshort[90:100])

# file = open(claudiasbaselinerep+"RMSEs/"+"CBtypeRMSE_"+str(N_unique)+".txt", "w")
# file.write("h1, h2, h3, h4, FPRMSE, pixRMSE"+'\n')
# np.savetxt(file, index2_partitioned_baselines, delimiter=', ', fmt=fmt)
# file.close()

#16 files started approx 17:30 
#for all combs, 2 fps, 2 pix calcs

In [ ]:
import os
#os.path.basename('/folderA/folderB/folderC/folderD')
rmsepath = '/home/james/mylibs/multifrequency/baseline_files/ClaudiasBaselines/RMSEs/'
rmseApath = '/home/james/mylibs/multifrequency/baseline_files/ClaudiasBaselines/RMSEs/analysisdata/'

def RMSEcalc(file):
    #load file
    dat = np.loadtxt(file, delimiter=',', skiprows=1)
    
    if len(dat.shape) == 1:
        print("break for baseline of 1")
        return

    bi = np.linspace(0, len(dat[:,0])-1, len(dat[:,0]), dtype=int)
    comb = np.array(list(combinations(bi, 2)))
    
    #open a file here and file orginal file
    fname = os.path.basename(file)  
    sfile = open(rmsepath+'RMSE_'+fname, "w")
    sfile.write("Orginal CB type file -> "+file+'\n')
    sfile.write("tdh1, tdh2, tdh3, tdh4, h1, h2, h3, h4, rmse"+'\n')
    
    for i in range(len(comb[:,0])):
        print(i, comb[i], comb[i,0], comb[i,1], 
              dat[comb[i,0], 0], dat[comb[i,0], 1], dat[comb[i,1], 0], dat[comb[i,1], 1],
             int(FIhorns[np.where(TDhornsFIconf == dat[int(comb[i,0]),0])]),
             int(FIhorns[np.where(TDhornsFIconf == dat[int(comb[i,0]),1])]),
             int(FIhorns[np.where(TDhornsFIconf == dat[int(comb[i,1]),0])]),
             int(FIhorns[np.where(TDhornsFIconf == dat[int(comb[i,1]),1])]))
    
    #print(i)
        x, y, i1 = calculate_intensity_4_baseline((int(FIhorns[np.where(TDhornsFIconf == dat[int(comb[i,0]),0])]), int(FIhorns[np.where(TDhornsFIconf == dat[int(comb[i,0]),1])])), my150dat)
        x, y, i2 = calculate_intensity_4_baseline((int(FIhorns[np.where(TDhornsFIconf == dat[int(comb[i,1]),0])]), int(FIhorns[np.where(TDhornsFIconf == dat[int(comb[i,1]),1])])), my150dat)
        rmse1 = RMSE(i1/max(i1), i2/max(i2))
#         px, py, pi1 = IntegrateHornCombOnFP(i1, np.array([x, y]), vtxs)
#         px, py, pi2 = IntegrateHornCombOnFP(i2, np.array([x, y]), vtxs)
#         rmse2 = RMSE(pi1, pi2)
        sfile.write("{}, {}, {}, {}, {}, {}, {}, {}, {:.5e} \n".format(
            int(dat[comb[i,0], 0]),
            int(dat[comb[i,0], 1]),
            int(dat[comb[i,1], 0]),
            int(dat[comb[i,1], 1]),
            int(FIhorns[np.where(TDhornsFIconf == dat[int(comb[i,0]),0])]),
            int(FIhorns[np.where(TDhornsFIconf == dat[int(comb[i,0]),1])]),
            int(FIhorns[np.where(TDhornsFIconf == dat[int(comb[i,1]),0])]),
            int(FIhorns[np.where(TDhornsFIconf == dat[int(comb[i,1]),1])]),
            rmse1))
        
    #close file here
    sfile.close()
    
    return 



here the parallel code is run to calculate the RMSEs for all combinations in all combination types

In [ ]:
num_cores=16
inputs=tqdm(cbfiles)

startm = timer()

if __name__ == '__main__':
        process=Parallel(n_jobs=num_cores)(delayed(RMSEcalc)(file) for file in inputs)
        
endm = timer()
print("time taken parrallel", endm - startm, "s")

In [ ]:
"""95 - 108 10 seconds with no TES
85 - 108 24 seconds with no TES
85 - 108 with TES calc = 
55 mins for all combs and files with no TES rmse calc
"""

do the plot

In [ ]:

rmsepath = '/home/james/mylibs/multifrequency/baseline_files/ClaudiasBaselines/RMSEs/'
jet= plt.get_cmap('jet')
colors = iter(jet(np.linspace(0,10,11)))

rmsefiles = glob.glob(rmsepath+'*.txt')
rmsefiles.sort(key=lambda f: int(re.sub('\D', '', f)))
#print(rmsefiles)

typelist = np.array([])
rmses = np.array([])
rmsesm = np.array([])
rmsesSTD = np.array([])
ind = np.array([])

font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 20}
plt.rc('font', **font)
%matplotlib inline
plt.rc('font', **font)
plt.figure(figsize=(16, 8))

temp=0
#106 since only one baseline for these types
for i, file in enumerate(rmsefiles[0:106]):
    
    num = re.findall(r'\d+', file)
    #print(file, num, i)
    
    dat = np.loadtxt(file, delimiter = ',', skiprows=2)
    
    temp = len(dat[:,8].shape) + temp
    
    index = i+temp
        
    nums = np.full(dat[:,8].shape, num)
    
    #print(dat.shape, i, dat[:,8].shape, nums.shape)
    
    
    rmses = np.append(rmses, dat[:,8])
    rmsesm = np.append(rmsesm, np.mean(dat[:,8]))
    rmsesSTD = np.append(rmsesSTD, np.std(dat[:,8]))
    typelist = np.append(typelist, nums)
    ind = np.append(ind, index)

    plt.plot(nums, dat[:,8], '.', markersize=10)

plt.legend(loc='upper right', fontsize=12)
plt.xlabel('Baseline Type')
plt.ylabel('RMSE')

#plt.savefig('/media/james/DATA/baseline_figures/results_cb_all.png', facecolor='white')

plt.figure(figsize=(16, 8))
for i, file in enumerate(rmsefiles[0:106]):
    plt.plot(i+1, rmsesm[i], 's', markersize=5)
    
plt.gca().set_prop_cycle(None)
for i, file in enumerate(rmsefiles[0:106]):
    plt.errorbar(i+1, rmsesm[i], yerr=rmsesSTD[i], elinewidth=3)
    
plt.xlabel('Baseline Type')
plt.ylabel('Mean RMSE with Standard Deviation')

#plt.savefig('/media/james/DATA/baseline_figures/results_cb_meanstd.png', facecolor='white')

In [ ]:
"""finished"""

The rest is just test code and debugging that may be useful in the future

In [ ]:
print(len(typelist), len(rmses), len(ind))



In [ ]:
from matplotlib.pyplot import cm
import seaborn as sns
color=iter(cm.rainbow(np.linspace(0,len(typelist),len(typelist))))
colors = iter(plt.cm.Spectral(np.linspace(0,len(typelist),len(typelist))))
#print(colors.shape)

colors = sns.palplot(sns.color_palette("hls", len(typelist)))

color=iter(cm.rainbow(np.linspace(0,1,len(typelist))))
# for i in range(n):
#    cn = next(color)
#    plt.plot(x, y,c=c)


plt.figure(figsize=(16, 8))

plt.scatter(typelist, rmses, '.', color=colors)

#for i in range(len(typelist)):
    

# # # #     plt.plot(typelist[i], rmses[i], '.', 
# # # #              color=(0, typelist[i] / max(typelist), typelist[i] / max(typelist), typelist[i] / max(typelist)))
#     plt.plot(typelist[i], rmses[i], color=c)

#     if typelist[i] > typelist[i-1]:
#         c=next(color)
    

In [ ]:
# startm = timer()
# ls = []
# for file in cbfiles:
#     ls.append(RMSEcalc(file))

# endm = timer()
# print("time taken cpu loop", endm - startm, "s")

In [ ]:
"""okay great so parrallel is faster this time!"""

In [ ]:
# def RMSEcalc2(horns):
#     #print(horns)

#     #print(i)
#     x, y, i1 = calculate_intensity_4_baseline((121, 147), my150dat)
#     x, y, i2 = calculate_intensity_4_baseline((120, 146), my150dat)
#     rmse = RMSE(i1,i2)
#     print(rmse)
#     #px, py, pi = IntegrateHornCombOnFP(i, np.array([x, y]), vtxs)
#     #print(x.shape, i.shape, px.shape, pi.shape)
#     return rmse

In [ ]:
# """now try with file as main loop but combs as parrallel"""

# """now convert TD to FI and check 50, 51 has a problem type 97"""
# for file in cbfiles[0:1]:
#     print(file)
#     dat = np.loadtxt(file, delimiter=',', skiprows=1)
#     #print(len(dat.shape))
    
#     if len(dat.shape) == 1:
#         print("break for baseline of 1")
#         continue
    
#     bi = np.linspace(0, len(dat[:,0])-1, len(dat[:,0]), dtype=int)
#     arsize = scipy.special.factorial(len(bi)) / ( scipy.special.factorial(2) * scipy.special.factorial(len(bi) - 2))
#     #print("file, dat, bi, arsize, ", file, dat.shape, bi, arsize)
#     comb = np.array(list(combinations(bi, 2)))
#     print(comb.shape)
    
#     h1 = []
#     h2 = []
#     h3 = []
#     h4 = []
    
#     for i in range(len(comb[:,0])):
    
#         h1.append(FIhorns[np.where(TDhornsFIconf == dat[int(comb[i,0]),0])])
#         h2.append(FIhorns[np.where(TDhornsFIconf == dat[int(comb[i,0]),1])])
#         h3.append(FIhorns[np.where(TDhornsFIconf == dat[int(comb[i,1]),0])])
#         h4.append(FIhorns[np.where(TDhornsFIconf == dat[int(comb[i,1]),1])])
    
#     ha = np.hstack([h1, h2, h3, h4])
#     print(ha.shape)

# #     num_cores = 16

# #     if len(ha[:,0]) <= 16:
# #         num_cores=1
# #         continue
#     inputs=tqdm(ha)
    
    
#     if __name__ == '__main__':
#         process=Parallel(n_jobs=num_cores)(delayed(RMSEcalc2)(i) for i in inputs)
        
#     #RMSE for these combinations
#     RMSE = np.array(process)
#     print("RMSE shape, ", RMSE.shape)
    
#     tosave = np.hstack([ha, RMSE])
#     print("to save shape,  ", tosave.shape)

In [ ]:
# #print(len(inputs))
# print(ha.shape)

In [ ]:
# for i in ha:
#     print(i)

In [ ]:
# inputs=tqdm(ha)

# startm = timer()
    
# if __name__ == '__main__':
#     proces=Parallel(n_jobs=num_cores)(delayed(RMSEcalc2)(i) for i in inputs)

        
# endm = timer()
# print("time taken parrallel", endm - startm, "s")

In [ ]:
# print(len(proces), proces)

In [ ]:
"""time to loop over 378 horn combs a calc rmse residual approx 32 seconds"""

In [ ]:
# inputs=tqdm(ha)

In [ ]:
# print(inputs)

In [ ]:
def RMSEcalc3(h1, h2, h3, h4):
    #print(horns)

    #print(i)
    x, y, i1 = calculate_intensity_4_baseline((h1, h2), my150dat)
    x, y, i2 = calculate_intensity_4_baseline((h3, h4), my150dat)
    rmse1 = RMSE(i1, i2)
    print(rmse)
    px, py, pi1 = IntegrateHornCombOnFP(i, np.array([x, y]), vtxs)
    px, py, pi2 = IntegrateHornCombOnFP(i, np.array([x, y]), vtxs)
    rmse2 = RMSE(pi1, pi2)
    #print(x.shape, i.shape, px.shape, pi.shape)
    return rmse1, rmse2

In [ ]:
def RMSEforHornCombs(file):
    print(file)
    #load file
    dat = np.loadtxt(file, delimiter=',', skiprows=1)
    #print(len(dat.shape))
    
    #check to make sure combination exists
    if len(dat.shape) == 1:
        print("break for baseline of 1")
        return
        
    #create indexed combinations
    bi = np.linspace(0, len(dat[:,0])-1, len(dat[:,0]), dtype=int)
    #arsize = scipy.special.factorial(len(bi)) / ( scipy.special.factorial(2) * scipy.special.factorial(len(bi) - 2))
    #print("file, dat, bi, arsize, ", file, dat.shape, bi, arsize)
    comb = np.array(list(combinations(bi, 2)))
    print(comb.shape)
    
    h1 = np.array(len(comb[:,0]))
    h2 = np.array(len(comb[:,0]))
    h3 = np.array(len(comb[:,0]))
    h4 = np.array(len(comb[:,0]))
    #set TD horns in FI indexing
    h1 = np.array(FIhorns[np.where(TDhornsFIconf == dat[int(comb[:,0]),0])])
    h2 = FIhorns[np.where(TDhornsFIconf == dat[int(comb[:,0]),1])]
    h3 = FIhorns[np.where(TDhornsFIconf == dat[int(comb[:,1]),0])]
    h4 = FIhorns[np.where(TDhornsFIconf == dat[int(comb[:,1]),1])]
    
    #calculation RMSE for each horn combination
    RMSE_FP = []
    RMSE_dets = []
    for i in range(len(comb[:,0])):
    
        rmse1, rmse2 = RMSEcalc3(h1[i], h2[i], h3[i], h4[i])
        
        RMSE_FP.append(rmse1)
        RMSE_dets.append(rmse2)
        
    RMSE_FP = np.array(RMSE_FP)
    RMSE_dets = np.array(RMSE_dets)
    
    savear = np.hstack([h1, h2, h3, h4, RMSE_FP, RMSE_dets])
    print(savear.shape)
    
    #now save to file
    
    return

In [ ]:
num_cores=16
inputs=tqdm(cbfiles)

startm = timer()

if __name__ == '__main__':
        process=Parallel(n_jobs=num_cores)(delayed(RMSEforHornCombs)(file) for file in inputs)
        
endm = timer()
print("time taken parrallel", endm - startm, "s")